In [1]:
# setup

import pika
import time

# RabbitMQ server details
rabbitmq_host = 'localhost' 
queue_name = 'temperature_queue'


frequency = 3 #PT sends data every (frequency) seconds
csv_file = "temperature_readings.csv"


# Setup RabbitMQ connection and channel
connection = pika.BlockingConnection(pika.ConnectionParameters(host=rabbitmq_host))
channel = connection.channel()
channel.queue_declare(queue=queue_name, durable=True)

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=10', 'queue=temperature_queue'])>"])>

In [2]:
# main loop

with open(csv_file, 'r') as file:
    # Skip the header row
    next(file)
    
    timestep = 0 # initial timestamp in minutes
    
    # Loop through the csv file line by line
    for line in file:
        line = line.strip()
        columns = line.split(",")
        sensor_data = ', '.join(columns)
        
        # The message includes the timestep and all the sensors readings
        message = f"Timestep: {timestep} min, Sensor Data: {sensor_data}"
        
        # Send the message to the RabbitMQ queue
        channel.basic_publish(
            exchange='',  
            routing_key=queue_name, 
            body=message, 
            properties=pika.BasicProperties(
                delivery_mode=2,
            )
        )
        
        print(f"Sent: {message}")
        timestep += 30 # next readings happen after 30 minutes
        time.sleep(frequency)


Sent: Timestep: 0 min, Sensor Data: 4.009245487837425514e+00, -1.012926203433399008e+00, 1.766810008025754497e+00, 3.758027376754904836e+00, 4.220252961921182155e+00, 4.128572701526185718e+00, 4.015375034783984276e+00, 3.993660480081748787e+00, 3.987142135950724420e+00, 3.997497643202365225e+00, 4.000195019586271883e+00
Sent: Timestep: 30 min, Sensor Data: 6.089447984676369607e+00, -5.088872303318858759e-01, 1.684593569089609222e+00, 3.634665365761029499e+00, 4.184457520915994877e+00, 4.130877667058367564e+00, 4.030288583117103585e+00, 3.987070233338603220e+00, 4.001751205369838083e+00, 4.000057348378115663e+00, 3.997727337725588193e+00
Sent: Timestep: 60 min, Sensor Data: 8.148684491142610398e+00, 7.215485681099323423e-02, 1.641808432094043591e+00, 3.538871406438668377e+00, 4.131746493766657480e+00, 4.123083422111172958e+00, 4.047853865614636781e+00, 3.995894207712603841e+00, 3.998145133562999831e+00, 4.007800387139379694e+00, 4.004628937406044642e+00
Sent: Timestep: 90 min, Sensor Da

KeyboardInterrupt: 